# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [15]:
import requests
import json

from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [10]:
# workspace setup
ws = Workspace.from_config()
print(ws)

# dataset key info
key = 'nba_3pt_data'
data_desc = 'NBA game data from 2015 to 2023 containing 3 point percentages.'

# checking if dataset is already registered
if key in ws.datasets.keys():
    nba_dataset = ws.datasets(key)
else:
    # get data from url
    nba_data_url = 'https://raw.githubusercontent.com/mattwatson50/udacity_azure_ml_final/main/data/nba_3pt_data.csv'
    dataset = Dataset.Tabular.from_delimited_files(nba_data_url)        
    # register dataset
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=data_desc)

# getting nba dataset
nba_df = nba_dataset.to_pandas_dataframe()
print(nba_df.head())

# choose a name for experiment
experiment_name = 'automl_nba_3pt'

experiment=Experiment(ws, experiment_name)

Workspace.create(name='quick-starts-ws-256073', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-256073')
   season_id matchup_home     season_type  min         team_name_home  \
0      22015  ATL vs. DET  Regular Season  240          Atlanta Hawks   
1      22015  GSW vs. NOP  Regular Season  240  Golden State Warriors   
2      22015  CHI vs. CLE  Regular Season  240          Chicago Bulls   
3      22015  BOS vs. PHI  Regular Season  240         Boston Celtics   
4      22015  MIA vs. CHA  Regular Season  240             Miami Heat   

   fg3m_home  fg3a_home  fg3_pct_home wl_home        team_name_away  \
0          8         27         0.296       L       Detroit Pistons   
1          9         30         0.300       W  New Orleans Pelicans   
2          7         19         0.368       W   Cleveland Cavaliers   
3          8         24         0.333       W    Philadelphia 76ers   
4         12         20         0.600       W     Charlotte H

In [11]:
# creating a compute cluster
cluster_name = "proj-compute-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster already created.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=6)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Cluster already created.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [12]:
# TODO: Put your automl settings here
automl_settings = {
    'experiment_timeout_minutes' : 30,
    'task' : 'classification',
    'primary_metric' : 'accuracy',
    'training_data' : nba_dataset,
    'label_column_name' : 'highest_3pp_wl',
    'n_cross_validations' : 5,
    'enable_early_stopping' : True,
    'enable_onnx_compatible_models' : True
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cluster_name,
                             **automl_settings)

In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_nba,AutoML_9f2f0b89-96e9-4386-b51d-4f96314aef42,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
# showing RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [31]:
# Retrieve and save your best automl model.

# getting best model and saving it out with onnx
automl_run, automl_best_model = remote_run.get_output(return_onnx_model=True)
print('Run ID: ' + automl_run.id)
print('Model Name: ' + automl_run.properties['model_name'])
print('Model Details:')
print(automl_best_model)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [32]:
#TODO: Save the best model

# convert and save the model
OnnxConverter.save_onnx_model(automl_best_model, file_path="./automl_best_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# register the model
model_name = 'nba_3pt_voting_ensemble'
model_description = 'Voting Ensemble Model for NBA 3PT Percentage Win Analysis'
registered_model = automl_run.register_model(model_name=model_name, description=model_description)

# creating inference config and deploying the model as a web service
service_name = 'nba-data-env-service'

# creating environment
nba_env = Environment(name="nba-env")
nba_env.register(workspace=ws)

inference_config = InferenceConfig(entry_script='score.py',environment=nba_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[automl_best_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = "http://3791612e-f3ba-4abc-927d-c92ae5ff6aea.westus2.azurecontainer.io/score"

# If the service is authenticated, set the key or token
key = "8fQomtrNOLgHDISBo3tZoOMrtnREHaz0"

# Two sets of data to score, so we get two results back
data = {
    "Inputs": 
	{
	"data":
	[
      {
        'season_id': 22015,
        'matchup_home': 'ATL vs. DET',
        'season_type': 'Regular Season',
        'min': 240,
        'team_name_home': 'Atlanta Hawks',
        'fg3m_home': 8,
        'fg3a_home': 27,
        'fg3_pct_home': 0.296,
        'wl_home': 'L',
        'team_name_away': 'Detroit Pistons',
        'fg3m_away': 12,
        'fg3a_away': 29,
        'fg3_pct_away': 0.414,
        'wl_away': 'W',
        'highest_3pp': 0.414,
        'highest_3pp_wl': 'W'
      },
   ]
}
}
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {"Content-Type": "application/json"}
# If authentication is enabled, set the authorization header
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
